In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Load the dataset
data = pd.read_csv('/Users/shacheesb/Downloads/jigsaw-toxic-comment-train.csv')  # Replace with your dataset path

# Check for missing values and drop them
data = data.dropna()

# Extract features and labels
texts = data['comment_text']
labels = data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

# Tokenize the text
tokenizer = Tokenizer(num_words=20000)  # Limit to top 20,000 words
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print(f"Found {len(word_index)} unique tokens.")

# Pad sequences
max_sequence_length = 200
data_padded = pad_sequences(sequences, maxlen=max_sequence_length)

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(data_padded, labels, test_size=0.2, random_state=42)

# Define the GRU model
model = Sequential([
    Embedding(input_dim=20000, output_dim=128, input_length=max_sequence_length),
    Bidirectional(GRU(64, return_sequences=True)),
    Dropout(0.3),
    Bidirectional(GRU(32)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(6, activation='sigmoid')  # Sigmoid for multi-label classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=64,
    verbose=1
)

# Evaluate the model
val_predictions = model.predict(X_val)
val_predictions = (val_predictions > 0.5).astype(int)

# Classification report
print(classification_report(y_val, val_predictions, target_names=labels.columns))


Found 300257 unique tokens.
Epoch 1/5


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


2795/2795 ━━━━━━━━━━━━━━━━━━━━ 8327s 3s/step - accuracy: 0.9159 - loss: 0.0907 - val_accuracy: 0.9951 - val_loss: 0.0532
Epoch 2/5
2795/2795 ━━━━━━━━━━━━━━━━━━━━ 1177s 421ms/step - accuracy: 0.9941 - loss: 0.0499 - val_accuracy: 0.9942 - val_loss: 0.0511
Epoch 3/5
2795/2795 ━━━━━━━━━━━━━━━━━━━━ 1103s 395ms/step - accuracy: 0.9917 - loss: 0.0431 - val_accuracy: 0.9938 - val_loss: 0.0540
Epoch 4/5
2795/2795 ━━━━━━━━━━━━━━━━━━━━ 2154s 771ms/step - accuracy: 0.9887 - loss: 0.0373 - val_accuracy: 0.9949 - val_loss: 0.0558
Epoch 5/5
2795/2795 ━━━━━━━━━━━━━━━━━━━━ 8940s 3s/step - accuracy: 0.9614 - loss: 0.0324 - val_accuracy: 0.9905 - val_loss: 0.0577
1398/1398 ━━━━━━━━━━━━━━━━━━━━ 66s 47ms/step
               precision    recall  f1-score   support

        toxic       0.74      0.76      0.75      4266
 severe_toxic       0.65      0.12      0.20       384
      obscene       0.77      0.77      0.77      2486
       threat       0.55      0.23      0.32       133
       insult       0.73 

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [6]:
# Function to preprocess and predict toxic categories
def predict_toxicity(comment):
    # Tokenize and pad the input comment
    sequence = tokenizer.texts_to_sequences([comment])
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length)
    
    # Predict the toxic categories
    prediction = model.predict(padded_sequence)
    
    # Convert predictions to binary (threshold = 0.5)
    prediction_binary = (prediction > 0.5).astype(int)
    
    # Display results
    categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    for category, pred in zip(categories, prediction_binary[0]):
        print(f"{category}: {'Yes' if pred == 1 else 'No'}")

# Example comment to test
test_comment = "You're such a horrible person!"
predict_toxicity(test_comment)
test_comment1 = "You should go to hell!"
predict_toxicity(test_comment1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
toxic: Yes
severe_toxic: No
obscene: No
threat: No
insult: No
identity_hate: No
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
toxic: Yes
severe_toxic: No
obscene: No
threat: No
insult: Yes
identity_hate: No
